In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

# グラフのグローバル設定
plt.rcParams['font.size'] = 13
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['xtick.labelsize'] = 13
plt.rcParams['ytick.labelsize'] = 13

# データの読み込み
file_path = 'data.csv'
# エンコーディングはデータに合わせて適宜変更 (shift_jis または utf-8)
try:
    df = pd.read_csv(file_path, encoding='shift_jis')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='utf-8')

# カラム名の定義とリネーム
column_rename_map = {
    'クラフトビール「白蒲田」をまた飲んでみたいと思いますか？': 'リピート意向',
    '甘味をどの程度感じましたか？（１／５）': '甘味',
    'うま味をどの程度感じましたか？（２／５）': 'うま味',
    '苦味をどの程度感じましたか？（３／５）': '苦味',
    '酸味をどの程度感じましたか？（４／５）': '酸味',
    '塩味をどの程度感じましたか？（５／５）': '塩味',
    '華やかさをどの程度感じましたか？（１／５）': '華やかさ',
    '開放感をどの程度感じましたか？（２／５）': '開放感',
    '爽やかさをどの程度感じましたか？（３／５）': '爽やかさ',
    '賑い感をどの程度感じましたか？（４／５）': '賑わい感',
    '賑わい感をどの程度感じましたか？（４／５）': '賑わい感',
    '高揚感をどの程度感じましたか？（５／５）': '高揚感',
    '年齢を教えてください。': '年齢',
    '性別を教えてください。': '性別'
}

# グラフタイトル用のカテゴリ区分
category_map = {
    'リピート意向': '（目的変数）',
    '甘味': '（味覚）', 'うま味': '（味覚）', '苦味': '（味覚）', '酸味': '（味覚）', '塩味': '（味覚）',
    '華やかさ': '（視覚）', '開放感': '（視覚）', '爽やかさ': '（視覚）', '賑わい感': '（視覚）', '高揚感': '（視覚）',
    '年齢': '（属性）', '性別': '（属性）'
}

# 必要な列のみ抽出してリネーム
valid_columns = [col for col in column_rename_map.keys() if col in df.columns]
df_plot = df[valid_columns].rename(columns=column_rename_map)

# プロット対象の項目リスト（辞書の値の順序を保持しつつ重複除去）
plot_items = sorted(set(df_plot.columns) & set(column_rename_map.values()), key=list(column_rename_map.values()).index)

# --- データ型変換 ---
# 年齢と評価項目を数値に変換（性別以外）
numeric_columns = [col for col in plot_items if col != '性別']
for col in numeric_columns:
    df_plot[col] = pd.to_numeric(df_plot[col], errors='coerce')

# --- グラフの描画 ---
# グリッド設定 (4列 x N行)
n_cols = 4
n_rows = int(np.ceil(len(plot_items) / n_cols))
eval_scores = range(6) # 0〜5点の評価軸

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 4 * n_rows), constrained_layout=True)
axes = axes.flatten()

# タイトル設定
sample_size = len(df_plot.dropna(how='all'))
fig.suptitle(f'各項目の回答分布 (n={sample_size})', fontsize=24)

# 各項目をループ処理で描画
for i, item_name in enumerate(plot_items):
    ax = axes[i]
    category_label = category_map.get(item_name, '')
    ax.set_title(f"{item_name}{category_label}")
    ax.set_ylabel('回答者数 (人)')
    
    # 項目タイプ別の描画処理
    if item_name == '年齢':
        ax.hist(df_plot[item_name].dropna(), bins=10, color='mediumseagreen', edgecolor='black')
        ax.set_xlabel('年齢')
        
    elif item_name == '性別':
        counts = df_plot[item_name].value_counts()
        ax.bar(counts.index, counts.values, color='goldenrod', edgecolor='black')
        ax.set_xlabel('性別')
        
    else:
        # 評価項目 (0-5点)
        counts = df_plot[item_name].value_counts().reindex(eval_scores, fill_value=0)
        ax.bar(counts.index, counts.values, color='royalblue', edgecolor='black')
        ax.set_xlabel('評価点')
        ax.set_xticks(eval_scores)

# 余ったサブプロットを非表示
for i in range(len(plot_items), len(axes)):
    axes[i].axis('off')

# 保存と表示
output_filename = 'figure_distribution.png'
plt.savefig(output_filename, dpi=300, bbox_inches='tight')
plt.show()